In [ ]:
!pip install hyperopt

In [ ]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [ ]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

In [5]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [6]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [7]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [9]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
  )
  
  return model

In [10]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [24]:
def get_cnn_v5(input_shape, num_classes): 
  return Sequential([
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'), 
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Flatten(),

      Dense(1024, activation='relu'), 
      Dropout(0.3),

      Dense(1024, activation='relu'), 
      Dropout(0.3),

      Dense(num_classes, activation='softmax'),              
  ])

In [25]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.4633 - accuracy: 0.3137 - val_loss: 0.7349 - val_accuracy: 0.7568
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.6240 - accuracy: 0.8053 - val_loss: 0.1268 - val_accuracy: 0.9640
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2842 - accuracy: 0.9146 - val_loss: 0.0613 - val_accuracy: 0.9818
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1981 - accuracy: 0.9402 - val_loss: 0.0356 - val_accuracy: 0.9894
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1552 - accuracy: 0.9563 - val_loss: 0.0268 - val_accuracy: 0.9928


0.9782312925170068

In [31]:
def get_model(params): 
  return Sequential([
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'), 
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),

      Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
      Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),

      Flatten(),

      Dense(1024, activation='relu'), 
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation='relu'), 
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax'),              
  ])

In [36]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [37]:
space={
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('ddropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.31486096734343555, 'dropout_cnn_block_three': 0.34433744321205023, 'dropout_cnn_block_two': 0.41373477834027633, 'dropout_dense_block_one': 0.38569015853282096, 'dropout_dense_block_two': 0.442587451919424}
accuracy=0.9476190209388733
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3638330852592357, 'dropout_cnn_block_three': 0.3465573465109629, 'dropout_cnn_block_two': 0.43402637642622377, 'dropout_dense_block_one': 0.40329044406338727, 'dropout_dense_block_two': 0.48937681929366345}
accuracy=0.9215419292449951
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.451664315564418, 'dropout_cnn_block_three': 0.4295398129600363, 'dropout_cnn_block_two': 0.4428893199150158, 'dropout_dense_block_one': 0.34319589213576807, 'dropout_dense_block_two': 0.32130742013570057}
accuracy=0.9698412418365479
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.49415718340036874, 'dropout_cnn_block_three': 0.38798063196402754, 'dropout_cnn_block_two': 0.4316068